In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
/
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x  #gpu
except Exception:
  pass
import tensorflow as tf

import cProfile

# 导入TF2.0


In [3]:
tf.executing_eagerly()

True

使用tf进行简单的矩阵计算

In [6]:
x = [[2.]]
m = tf.matmul(x,x)
print("x matmul x = {}".format(m))

x matmul x = [[4.]]


创建常量

In [7]:
a = tf.constant([[1,2],
                 [3,4]])
print(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [8]:
b = tf.add(a,1)#参数都相加 1
print(b)

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


In [9]:
print(a*b)

tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


In [10]:
print(tf.matmul(a,b))#矩阵a乘以矩阵b  ，和tf.multiply（a，b）进行区别：两个矩阵中对应元素各自相乘

tf.Tensor(
[[10 13]
 [22 29]], shape=(2, 2), dtype=int32)


使用Numpy进行匹配

In [11]:
import numpy as np #导入

In [12]:
c = np.multiply(a,b)
print(c)

[[ 2  6]
 [12 20]]


In [13]:
print(a.numpy())

[[1 2]
 [3 4]]


# 梯度计算

在实现机器学习算法时，如训练神经网络的反向传播，自动微分是很有效的。在执行时，使用tf.GradientTape来跟踪后面计算梯度的操作

In [14]:
w = tf.Variable([[1.0]])#初始化
with tf.GradientTape() as tape:
    loss = w * w
grad = tape.gradient(loss,w)
print(grad)

tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


# 训练模型

In [15]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()#下载数据集
x_train = x_train[:10000,:,:]
y_train = y_train[:10000]
x_test = x_test[:1000,:,:]
y_test = y_test[:1000]

11493376/11490434 [==============================] - 5s 0us/step


In [16]:
x_train = tf.cast(x_train[...,tf.newaxis]/255, tf.float32),
x_test = tf.cast(x_test[...,tf.newaxis]/255, tf.float32),

y_train = tf.keras.utils.to_categorical(y_train,10)
y_test = tf.keras.utils.to_categorical(y_test,10)

构建模型

In [17]:
mnist_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,[3,3],activation='relu',
                          input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(64,[3,3],activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10,activation="softmax")
])

In [20]:
mnist_model.summary()#输出模型各层的参数状况

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [21]:
inputs = tf.keras.Input(shape=(None,None,1),name="digits")
conv_1 = tf.keras.layers.Conv2D(16,[3,3],activation="relu")(inputs)
conv_2 = tf.keras.layers.Conv2D(16,[3,3],activation="relu")(conv_1)
ave_pool = tf.keras.layers.GlobalAveragePooling2D()(conv_2)
outputs = tf.keras.layers.Dense(10)(ave_pool)
mnist_model_2 = tf.keras.Model(inputs=inputs,outputs=outputs)

In [22]:
mnist_model_2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, None, None, 1)]   0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 16)    160       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 16)    2320      
_________________________________________________________________
global_average_pooling2d (Gl (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                170       
Total params: 2,650
Trainable params: 2,650
Non-trainable params: 0
_________________________________________________________________


 keras fit训练方法

In [23]:
mnist_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
                    validation_split=0.1,shuffle=True,
                   loss = tf.keras.losses.categorical_crossentropy,
                   metrics = ["accuracy"])

In [24]:
mnist_model_2.compile(loss = tf.keras.losses.categorical_crossentropy,
                      optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [25]:
mnist_model.fit(x_train,y_train,batch_size=128,epochs=3)

Train on 10000 samples
Epoch 1/3
10000/10000 [==============================] - 11s 1ms/sample - loss: 2.2937 - accuracy: 0.1291
Epoch 2/3
10000/10000 [==============================] - 10s 1ms/sample - loss: 2.2637 - accuracy: 0.1854
Epoch 3/3
10000/10000 [==============================] - 11s 1ms/sample - loss: 2.2340 - accuracy: 0.2451


In [26]:
mnist_model.evaluate(x_test,y_test)#输入数据和标签,输出损失和精确度   model.predict：输入测试数据,输出预测结果

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[2.2041776237487793, 0.473]

In [30]:
x_train[0]

<tf.Tensor: id=29, shape=(10000, 28, 28, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [

In [32]:
mnist_model.predict(x_test[0][1])

ValueError: Error when checking input: expected conv2d_input to have 4 dimensions, but got array with shape (28, 28, 1)

TF 2.0

In [33]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train[:10000,:,:]
y_train = y_train[:10000]
x_test = x_test[:1000,:,:]
y_test = y_test[:1000]

In [34]:
dataset = tf.data.Dataset.from_tensor_slices(
(tf.cast(x_train[...,tf.newaxis]/255, tf.float32),
 tf.cast(y_train,tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

In [35]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_history = []

In [36]:
for epoch in range(5):
    
    for (batch, (images,labels)) in enumerate (dataset):
        
        with tf.GradientTape() as tape:
            
            logits = mnist_model(images,training=True)
            loss_value = loss(labels,logits)
            
        grads = tape.gradient(loss_value,mnist_model.trainable_variables)
        optimizer.apply_gradients(zip(grads,mnist_model.trainable_variables))
        
    print("Epoch {} finishted".format(epoch))

Epoch 0 finishted
Epoch 1 finishted
Epoch 2 finishted
Epoch 3 finishted
Epoch 4 finishted


# 简答

1. Compared to FNN, what is the biggest advantage of CNN?

In [ ]:
CNN：比FNN更简单的网络结构，需要更少的参数
    学习更小的pattern
    只学习必要的pattern
    采样操作使得网络模型进一步简化

In [ ]:
RNN讲究序列，在时间维度上有个先后顺序，认为后续的值是有前者和一些参数的概率决定的，输入的顺序会影响输出
CNN讲究空间，强调的是空间中的窗口，从局部信息聚合得到整体信息，对输入进行层次信息提取，不仅需要考虑前后的序列问题还需要考虑空间的上下问题，

RNN：
1.RNN可以处理任意输入/输出长度。

2.与前馈神经网络不同，RNN可以使用其内部存储器来处理任意输入序列。

3.循环神经网络使用时间序列信息。 比如我上次发言的内容将影响我接下来要讲的内容。

4.RNN是文本和语音分析的理想选择。

CNN：
1.CNN采用固定大小的输入并生成固定大小的输出。

2.CNN是一种前馈人工神经网络 - 是多层感知器的变体，旨在使用最少量的预处理。

3.CNN使用其神经元之间的连接模式，并受到动物视觉皮层组织的启发，动物视觉皮层的个体神经元以这样的方式排列，即它们  响应于平铺视野的重叠区域。

4.CNN是图像和视频处理的理想选择。


2. Suppose your input is a 100 by 100 gray image, and you use a convolutional layer with 50 filters that are each 5x5. How many parameters does this hidden layer have (including the bias parameters)?

3. What are "local invariant" and "parameter sharing" ?

In [ ]:
参数共享：CNN是通过“卷积核”作为中介，同一个卷积核在所有图像内都是共享的，图像通过卷积操作后仍然保留原来的位置关系，一层卷积学到的特征往往是局部的，层数越高，学到的特征就越全局化，复合多个“卷积层”和“采样层”对输入信号进行加工，然后再连接层实现与输出目标之间的映射。
           卷积操作实际是在提取一个个局部信息，而局部信息的一些统计特性和其他部分是一样的，也就意味着这部分学到的特征也可以用到另一部分上。所以对图像上的所有位置，都能使用同样的学习特征
    
    
    
局部不变特征:局部特征的检测或描述对图像的各种变化，例如几何变化，光度变化，卷积变化
    通常存在一个局部的支撑邻域，可能是图像的任何子集，局部不变特征不仅能够在观测条件变化大，遮挡和杂乱干扰的情况下获取可靠的匹配，而且能够有效的描述图像内容进行图像检索或场景，目标识别。
    

4. Why we use batch normalization ?

In [ ]:
1.允许较大的学习率，加快模型收敛速度
    BN通过固定网络层输入（也即前一层的响应）的分布（标准正态）使优化过程中的解空间更平滑，从而确保了梯度更具预测性和稳定性
2.避免深层网络的梯度消失或爆炸问题
    BN通过固定网络层输入的均值和方差，即使网络较深层的响应或梯度过小或过大时，也可通过BN的规范化作用将其缩放到一个比较合理的取值范围
3.减少对参数初始化方法的依赖
    BN直接对网络层构造标准正态分布的响应,可以使网络各层的输入输出具有相同的统计分布

5. What problem does dropout try to solve ?

In [37]:
Dropout:在神经网络的训练过程中提出的一种防止过拟合的策略,旨在训练过程中按照一定的概率随机删除网络中的神经元.一般：隐藏层采样概率为0.5，输入层采样概率为0.8
    因为深度神经网络在悬链的过程中总会遇到容易过拟合和费时的问题


SyntaxError: invalid syntax (<ipython-input-37-7e232aa7628f>, line 1)

 6.Is the following statement correct and why ? "Because pooling layers do not have parameters, they do not affect the backpropagation(derivatives) calculation"

In [ ]:
不正确，pooling层没有需要学习的参数，但反向传播还是要考虑把1个像素的梯度传递给4个像素时需要保证传递的loss（或者梯度）总和不变
在max_pooling算法中：反向传播把梯度直接传给前一层某一个像素，而其他像素不接受梯度，也就是为0
但在mean pooling算法中：是把某个元素的梯度等分为n份分配给前一层，这样就保证池化前后的梯度（残差）之和保持不变